# YOLOv8 Firearm Detection — Colab Quickstart

This Colab trains a small YOLOv8 model for firearm detection.

**Steps:** Install deps → (Optional) Mount Drive → Prepare dataset → Train → Validate → Export → Test inference.

> ⚠️ Ethics: Use responsibly. Comply with local laws and privacy policies.


In [ ]:
#@title Install dependencies
!pip -q install ultralytics opencv-python onnxruntime
import ultralytics, sys
print('Ultralytics version:', ultralytics.__version__)
print(sys.version)


In [ ]:
#@title (Optional) Mount Google Drive
USE_DRIVE = False  #@param {type:"boolean"}
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = '/content/drive/MyDrive/yolov8-firearm'
else:
    BASE_DIR = '/content/yolov8-firearm'

import os
os.makedirs(BASE_DIR, exist_ok=True)
print('BASE_DIR =', BASE_DIR)


In [ ]:
#@title Dataset setup
# Provide your own dataset. This cell creates a tiny placeholder structure.
import os, yaml, pathlib
data_root = os.path.join(BASE_DIR, 'data')
os.makedirs(os.path.join(data_root, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(data_root, 'val', 'images'), exist_ok=True)

# Write data.yaml (edit 'train' and 'val' to your real paths if using your dataset)
data_yaml = {
    'train': os.path.join(data_root, 'train', 'images'),
    'val': os.path.join(data_root, 'val', 'images'),
    'nc': 1,
    'names': ['firearm']
}
yaml_path = os.path.join(BASE_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data_yaml, f)
print('Wrote', yaml_path)

print('➡️ Replace the placeholder with your dataset:')
print('  - Images go to .../train/images and .../val/images')
print('  - If using YOLO txt labels, put matching .txt files next to each image')


In [ ]:
#@title Train (edit hyperparameters as needed)
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # nano backbone for quick prototyping
results = model.train(data=yaml_path,
                      epochs=10,
                      batch=16,
                      imgsz=640,
                      project=os.path.join(BASE_DIR, 'runs', 'train'),
                      name='yolov8-firearm',
                      workers=2,
                      device=0 if ultralytics.__version__ else 0)
print('Training complete. Check runs/train/yolov8-firearm')


In [ ]:
#@title Validate
metrics = model.val(data=yaml_path, imgsz=640)
metrics


In [ ]:
#@title Export to ONNX
onnx_path = model.export(format='onnx', imgsz=640)
onnx_path


In [ ]:
#@title Test inference on a single image (provide path)
import cv2, numpy as np
test_img_path = ''  #@param {type:"string"}
weights = None  # if None, uses the last trained model; otherwise provide path to .pt
if weights is None:
    weights = os.path.join(BASE_DIR, 'runs', 'train', 'yolov8-firearm', 'weights', 'best.pt')
print('Using weights:', weights)
m = YOLO(weights)
if test_img_path:
    img = cv2.imread(test_img_path)
    res = m.predict(source=img, conf=0.35, verbose=False)
    r = res[0]
    print('Detections:', len(r.boxes))
else:
    print('⚠️ Provide test_img_path to run single-image inference')
